In [1]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix
import torch
import torch.nn as nn
import einops
from fancy_einsum import einsum
from tqdm.auto import tqdm
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import torch
import torch.nn as nn
import torch.nn.functional as F
from jaxtyping import Float
from functools import partial
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from typing import List, Tuple, Union
import os
import random
import rouge_score
from rouge_score import rouge_scorer
import os
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score
from copy import deepcopy

torch.set_grad_enabled(False)

def print_sys_info():
    import psutil
    import socket
    import gpustat
    memory = psutil.virtual_memory()
    print("剩余内存: {} G".format(memory.available/1024/1024//1024))
    host_name = socket.gethostname()
    print(f"当前主机名是:{host_name}")
    gpustat.print_gpustat()

def launch_clash():
    import subprocess
    import os

    result = subprocess.run("pidof clash", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if not result.stdout:
        subprocess.Popen("~/tools/clash/clash", shell=True)
        result = subprocess.run("pidof clash", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    print(f"Clash is running, pid: {result.stdout}")
    os.environ["http_proxy"] = "http://localhost:7890"
    os.environ["https_proxy"] = "http://localhost:7890"

print_sys_info()
launch_clash()

剩余内存: 851.0 G
当前主机名是:SH-IDC1-10-140-0-157
SH-IDC1-10-140-0-157      Fri Mar  8 11:18:46 2024  525.60.13
[0] NVIDIA A100-SXM4-80GB | 25°C,   0 % |     0 / 81920 MB |
[1] NVIDIA A100-SXM4-80GB | 45°C,  89 % | 53429 / 81920 MB | huyutao(53426M)
[2] NVIDIA A100-SXM4-80GB | 29°C,   0 % |  2423 / 81920 MB | yejin(2420M)
[3] NVIDIA A100-SXM4-80GB | 25°C,   0 % |     0 / 81920 MB |
[4] NVIDIA A100-SXM4-80GB | 32°C,  36 % | 28955 / 81920 MB | huyutao(28952M)
[5] NVIDIA A100-SXM4-80GB | 31°C,   0 % |     0 / 81920 MB |
[6] NVIDIA A100-SXM4-80GB | 39°C,   0 % |     0 / 81920 MB |
[7] NVIDIA A100-SXM4-80GB | 58°C, 100 % | 66843 / 81920 MB | yejin(66840M)
Clash is running, pid: 348131

time="2024-03-08T11:18:46+08:00" level=info msg="Start initial compatible provider 选择节点"
time="2024-03-08T11:18:46+08:00" level=info msg="Start initial compatible provider 主站加速"
time="2024-03-08T11:18:46+08:00" level=info msg="Start initial compatible provider 广告屏蔽"
time="2024-03-08T11:18:46+08:00" level=info msg="St

In [2]:
# hf_model_path = "google/gemma-2b-it"
hf_model_path = os.path.join(os.environ["my_models_dir"], "vicuna-7b-v1.1")
hf_tokenizer = AutoTokenizer.from_pretrained(hf_model_path)
hf_model = AutoModelForCausalLM.from_pretrained(hf_model_path)

model = HookedTransformer.from_pretrained("llama-7b-hf", dtype='bfloat16', hf_model=hf_model, tokenizer=hf_tokenizer, default_padding_side='left')

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/mnt/petrelfs/guoyiqiu/miniconda3/envs/mi/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Loaded pretrained model llama-7b-hf into HookedTransformer


In [3]:
dataset = load_dataset("allenai/sciq")

Using the latest cached version of the dataset since allenai/sciq couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /mnt/petrelfs/guoyiqiu/coding/huggingface/datasets/allenai___sciq/default/0.0.0/2c94ad3e1aafab77146f384e23536f97a4849815 (last modified on Fri Mar  8 03:48:47 2024).


time="2024-03-08T11:19:15+08:00" level=warning msg="[TCP] dial 选择节点 (match Match/) [::1]:6043 --> huggingface.co:443 error: forward1.10242018.xyz:6000 connect error: all DNS requests failed, first error: read udp4 10.140.0.157:2392->223.5.5.5:53: i/o timeout"


## Inplementation of internal state based uncertainty estimation  
- 使用一个统一的prompt，获取模型对训练集中的数据的预测结果  
	- prompt template  
- 选取一个RougeL阈值，将模型的预测结果分为正确和错误两类，计算正确率  
    - RougeL threshold
- 对正确和错误的预测结果，分别记录其内部状态，计算certainty vector 
    - pair selection: 
        - 随机选择等量的正确和错误的题目，使用模型生成的回复组成pair (可能有噪，即多生成了一些无关内容)
        - 随机选择等量的正确和错误的题目，使用标准答案和错误答案组成pair 
        - 选择模型生成后做对的题目，使用模型生成正确/错误答案时的句子组成pair
	- layer:
	    - 选择所有层的activation
	- token index
	    - 选择句子的最后一个token
	- activation
	    - 选择resid_post
	- clustering 
	    - PCA
- 使用相同的模板，获取模型在验证集上的预测结果，记录模型的内部状态  
- 使用验证集的预测结果和模型的内部状态，计算uncertainty score  
	- token idx
	    - 选择句子的最后一个token
- 使用uncertainty score和用RougeL计算出的模型的模型是否正确的标签，计算模型的AUROC

In [4]:
PROMPT_TEMPLATE = "Question:{q} Options:{o} Answer:"
TRAIN_SIZE = 1000
TEST_SIZE = 1000
ROUGE_L_THRESHOLD = 0.5

def get_dst_result(dst):
    results = []
    size = dst.num_rows
    for i in tqdm(range(size)):
        random.seed(i+42)
        options = [dst[i]["distractor1"], dst[i]["distractor2"], dst[i]["distractor3"], dst[i]["correct_answer"]]
        options_len = [len(model.to_str_tokens(opt, prepend_bos=False)) for opt in options]
        max_new_tokens = max(options_len)
        random.shuffle(options)
        prompt = PROMPT_TEMPLATE.format(q=dst[i]["question"], o=", ".join(options))
        output = model.generate(prompt, max_new_tokens=max_new_tokens, do_sample=False,verbose=False)
        answer = output.split("Answer:")[1].strip()
        print(output) if i == 0 else None
        results.append(dict(
            question=dst[i]["question"],
            options=options,
            answer=answer,
            correct=dst[i]["correct_answer"],
            input=prompt,
            output=output
        ))
    for res in results:
        scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        scores = scorer.score(res['answer'].lower(), res['correct'].lower())
        res["score"] = scores['rougeL']
    
    accuracy = sum([1 for res in results if res["score"].fmeasure > ROUGE_L_THRESHOLD])/len(results)
    print(f"Test Accuracy: {accuracy:.2%}")
    return results

train_dst = dataset["train"].select(range(TRAIN_SIZE))
train_result = get_dst_result(train_dst)

val_dst = dataset["validation"].select(range(TEST_SIZE))
val_result = get_dst_result(val_dst)

  0%|          | 0/1000 [00:00<?, ?it/s]

The following are multiple choice science exam questions about Physics, Chemistry and Biology.
 Question:What type of organism is commonly used in preparation of foods such as cheese and yogurt?
 Options:viruses, gymnosperms, mesophilic organisms, protozoa
 Answer:Bacteria

Question
Test Accuracy: 66.20%


  0%|          | 0/1000 [00:00<?, ?it/s]

The following are multiple choice science exam questions about Physics, Chemistry and Biology.
 Question:Who proposed the theory of evolution by natural selection?
 Options:Scopes, shaw, darwin, Linnaeus
 Answer:Charles Dar
Test Accuracy: 67.70%


In [5]:
CACHED_TOKEN_IDXS = -1
CACHED_ACT_NAME = 'resid_post'
CACHED_LAYERS = (0, model.cfg.n_layers)
CORRECT_PROMPT_TEMPLATE = "Question:{q}\n Options:{o}\n The correct answer is:"
INCORRECT_PROMPT_TEMPLATE = "Question:{q}\n Options:{o}\n The incorrect answer is:"


def get_cache(res_list, layers, token_idx):
    bsz = 16
    batched_correct_res = [res_list[i:i+bsz] for i in range(0, len(res_list), bsz)]
    all_cache = []
    for batched_res in tqdm(batched_correct_res):
        batched_input = [res['output'] for res in batched_res]
        out, cache = model.run_with_cache(batched_input, names_filter=lambda x: CACHED_ACT_NAME in x, device='cpu')
        batched_layer_cache = []
        for l in range(model.cfg.n_layers):
            batched_layer_cache.append(cache[utils.get_act_name(CACHED_ACT_NAME, l)].unsqueeze(0))
        batched_layer_cache = torch.cat(batched_layer_cache, dim=0).transpose(0,1) # [bsz layer pos d_model]
        all_cache.append(batched_layer_cache[:,layers[0]:layers[1],token_idx:,:])
    all_cache = torch.cat(all_cache, dim=0) # [bsz layer pos d_model]
    all_cache = all_cache # [bsz layer pos d_model]
    return all_cache.float()

def get_vector(res_list, layers, token_idx):
    correct_res = [res for res in deepcopy(res_list) if res["score"].fmeasure > ROUGE_L_THRESHOLD]
    for res in correct_res:
        # res['output'] = res['input'] + " " + res['correct']
        res['output'] = CORRECT_PROMPT_TEMPLATE.format(q=res['question'], o=", ".join(res['options'])) + " " + res['correct']
    print(f"Correct output: \n{correct_res[0]['output']}\n\nGT: {correct_res[0]['correct']}\n\n")
    
    incorrect_res = [res for res in deepcopy(res_list) if res["score"].fmeasure > ROUGE_L_THRESHOLD]
    for res in incorrect_res:
        wrong_options = [opt for opt in res['options'] if opt != res['correct']]
        # res['output'] = res['input'] + " " + random.choice(wrong_options)
        res['output'] = INCORRECT_PROMPT_TEMPLATE.format(q=res['question'], o=", ".join(res['options'])) + " " + random.choice(wrong_options)
        
    print(f"Incorrect output: \n{incorrect_res[0]['output']}\n\nGT: {incorrect_res[0]['correct']}\n\n")
    
    correct_cache = get_cache(correct_res, layers, token_idx)
    incorrect_cache = get_cache(incorrect_res, layers, token_idx)
    print(f"Correct Cache Shape: {correct_cache.shape}")
    print(f"Incorrect Cache Shape: {incorrect_cache.shape}")
    
    num_pair = min(correct_cache.shape[1], incorrect_cache.shape[0])
    diff_vector = correct_cache[:num_pair] - incorrect_cache[:num_pair] # [b l d]
    diff_vector = einops.rearrange(diff_vector, 'b l p d -> b (l p d)').cpu().float()
    pca = PCA(n_components=1)
    pca.fit(diff_vector)
    certainty_vector = torch.tensor(pca.components_[0], dtype=torch.float) # [layer d_model]
    certainty_vector = einops.rearrange(certainty_vector, '(l p d) -> l p d', p=abs(CACHED_TOKEN_IDXS), d=model.cfg.d_model) # [layer pos d_model]
    print(f"Certainty Vector Shape: {certainty_vector.shape}")
    return certainty_vector

In [6]:
def get_score_ours(val_result):
    certainty_vector = get_vector(train_result, CACHED_LAYERS, CACHED_TOKEN_IDXS)
    val_cache = get_cache(val_result, CACHED_LAYERS, CACHED_TOKEN_IDXS)
    print(f"Val Cache Shape: {val_cache.shape}")
    val_score = (einsum('b l p d, l p d -> b', val_cache, certainty_vector))
    val_score = val_score.tolist()
    # for res, score in zip(val_result, val_score):
    #     print(f"Question: {res['question']}\nOptions: {res['options']}\nAnswer: {res['answer']} \nCorrect answer: {res['correct']}\nRougeL: {res['score']}\nUncertainty Score: {score:.4f}\n")

    label = [1 if res["score"].fmeasure > ROUGE_L_THRESHOLD else 0 for res in val_result]

    auroc_score = roc_auc_score(label, val_score)
    print(f'AUROC: {auroc_score if auroc_score > 0.5 else 1 - auroc_score:.4f}')
    return auroc_score

ours_auroc_score = get_score_ours(val_result)

Correct output: 
The following are multiple choice science exam questions about Physics, Chemistry and Biology.
 Question:What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?
 Options:tropical effect, coriolis effect, centrifugal effect, muon effect
 The correct answer is: coriolis effect

GT: coriolis effect


Incorrect output: 
The following are multiple choice science exam questions about Physics, Chemistry and Biology.
 Question:What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?
 Options:tropical effect, coriolis effect, centrifugal effect, muon effect
 The incorrect answer is: centrifugal effect

GT: coriolis effect


  0%|          | 0/42 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

Correct Cache Shape: torch.Size([662, 32, 1, 4096])
Incorrect Cache Shape: torch.Size([662, 32, 1, 4096])
Certainty Vector Shape: torch.Size([32, 1, 4096])


  0%|          | 0/63 [00:00<?, ?it/s]

Val Cache Shape: torch.Size([1000, 32, 1, 4096])
AUROC: 0.7255


In [7]:
from sentence_transformers import SentenceTransformer, util
bert_model = SentenceTransformer('sentence-transformers/all-roberta-large-v1')

time="2024-03-08T11:27:01+08:00" level=warning msg="[TCP] dial 选择节点 (match Match/) [::1]:6247 --> huggingface.co:443 error: forward1.10242018.xyz:6000 connect error: all DNS requests failed, first error: Post \"https://rubyfish.cn/dns-query\": context deadline exceeded"
time="2024-03-08T11:27:06+08:00" level=warning msg="[TCP] dial 选择节点 (match Match/) [::1]:6249 --> huggingface.co:443 error: forward1.10242018.xyz:6000 connect error: all DNS requests failed, first error: read udp4 10.140.0.157:14053->223.5.5.5:53: i/o timeout"
time="2024-03-08T11:27:11+08:00" level=warning msg="[TCP] dial 选择节点 (match Match/) [::1]:6255 --> huggingface.co:443 error: forward1.10242018.xyz:6000 connect error: all DNS requests failed, first error: read udp4 10.140.0.157:36910->223.5.5.5:53: i/o timeout"
time="2024-03-08T11:27:16+08:00" level=warning msg="[TCP] dial 选择节点 (match Match/) [::1]:6257 --> huggingface.co:443 error: forward1.10242018.xyz:6000 connect error: all DNS requests failed, first error: rea

In [8]:
def get_score_sar(val_result):
    label = [1 if res["score"].fmeasure > ROUGE_L_THRESHOLD else 0 for res in val_result]
    answer_idxs = []
    for i,res in enumerate(val_result):
        start = model.get_token_position('Answer', res['output']) + 2
        end = len(model.to_str_tokens(res['output']))
        answer_idxs.append((start, end))
        print(f"Val Result[0]: {res['output']}\nTokens Generated: {model.to_str_tokens(res['output'])[start:end]}") if i == 0 else None
    print(f"answer_idxs[0]:{answer_idxs[0]}")
    bsz = 16
    batched_correct_res = [val_result[i:i+bsz] for i in range(0, len(val_result), bsz)]
    batched_answer_idxs = [answer_idxs[i:i+bsz] for i in range(0, len(answer_idxs), bsz)]
    all_sar_score = []
    all_sar_weights = []
    all_pe_score = []

    for batched_res, batched_idxs in tqdm(zip(batched_correct_res, batched_answer_idxs), total=len(batched_correct_res)):
        batched_input = [res['output'] for res in batched_res]
        logits = model(batched_input, return_type='logits', padding_side='right') # [bsz seq vocab]
        batched_orig_embedding = bert_model.encode(batched_input, convert_to_tensor=True)
        for i in range(len(batched_res)):
            input_string = batched_input[i]
            input_str_tokens = model.to_str_tokens(input_string)
            input_tokens = model.to_tokens(input_string)[0].tolist()
            assert len(input_str_tokens) == len(input_tokens)
            start, end = batched_idxs[i]
            prob = F.softmax(logits[i], dim=-1)
            # print(prob, prob.shape) if i == 0 and len(all_pe_score) == 0 else None
            logp = -torch.log(prob) # [len vocab]
            # print(prob[start:end].max(dim=-1)) if i == 0 and len(all_pe_score) == 0 else None
            pe = torch.tensor([logp[k-1][input_tokens[k]] for j,k in enumerate(range(start, end))], dtype=torch.float)
            # print(f'pe[0]: {pe.tolist()}') if i == 0 and len(all_pe_score) == 0 else None
            
            orig_embedding = batched_orig_embedding[i]
            new_input_strings = []
            for j in range(start, end):
                new_input_tokens = input_tokens[:j] + input_tokens[j+1:]
                new_input_string = model.to_string(new_input_tokens)
                # print(new_input_string) if j == start else None
                new_input_strings.append(new_input_string)
            new_input_strings_embedding = bert_model.encode(new_input_strings, convert_to_tensor=True)
            sim = util.cos_sim(orig_embedding, new_input_strings_embedding)[0].cpu()
            weights = 1 - sim
            weights = F.softmax(weights, dim=0)
            sar_score = einsum('s, s ->', pe, weights).item()
            pe_score = pe.mean().item()
            all_sar_score.append(sar_score)
            all_sar_weights.append(weights.tolist())
            all_pe_score.append(pe_score)
        # break
    print(f"all_sar_score[0]: {all_sar_score[0]} all_pe_score[0]: {all_pe_score[0]} weights[0]: {all_sar_weights[0]}")
    sar_auroc_score = roc_auc_score(label, all_sar_score)
    pe_auroc_score = roc_auc_score(label, all_pe_score)
    print(f'SAR AUROC: {sar_auroc_score if sar_auroc_score > 0.5 else 1 - sar_auroc_score:.4f}')
    print(f'PE AUROC: {pe_auroc_score if pe_auroc_score > 0.5 else 1 - pe_auroc_score:.4f}')
    return sar_auroc_score

sar_auroc_score = get_score_sar(val_result)

Val Result[0]: The following are multiple choice science exam questions about Physics, Chemistry and Biology.
 Question:Who proposed the theory of evolution by natural selection?
 Options:Scopes, shaw, darwin, Linnaeus
 Answer:Charles Dar
Tokens Generated: ['Char', 'les', 'Dar']
answer_idxs[0]:(50, 53)


  0%|          | 0/63 [00:00<?, ?it/s]

all_sar_score[0]: 0.25166264176368713 all_pe_score[0]: 0.251953125 weights[0]: [0.3325665295124054, 0.3303814232349396, 0.3370520770549774]
SAR AUROC: 0.6653
PE AUROC: 0.6655
